Cellule Python — diagnostics détaillés par fenêtre et heatmap par ville

 La cellule :

lit l’index enrichi results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index_enriched.csv ;

pour chaque fenêtre exporte un CSV diagnostics détaillés par feature (skew, median, mean, std, frac_NaN, top-3 valeurs extrêmes) et un CSV résumé par ville ;

crée un heatmap (cities × features numériques) sauvegardé en PNG (z-score ou min-max normalisé selon disponibilité) ;

sauvegarde les sorties dans results/temporal_cv_atypical_windows/diagnostics/<window_tag>/ et écrit des entrées de log dans logs/summary.md.

In [16]:
# Diagnostics détaillés et heatmaps par fenêtre atypique
import os, math, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.stats import skew

# Config paths
BASE_IDX = 'results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index_enriched.csv'
RAW_BASE_DIR = 'results/temporal_cv_atypical_windows'
OUT_BASE = os.path.join('results', 'temporal_cv_atypical_windows', 'diagnostics')
LOG_SUMMARY = os.path.join('logs', 'summary.md')
os.makedirs(OUT_BASE, exist_ok=True)
os.makedirs(os.path.dirname(LOG_SUMMARY), exist_ok=True)

def safe_tag(s):
    return re.sub(r'[^0-9A-Za-z_.-]', '_', str(s))

def append_log(msg):
    ts = datetime.utcnow().isoformat() + 'Z'
    with open(LOG_SUMMARY, 'a', encoding='utf-8') as f:
        f.write(f'- {ts} INFO: {msg}\n')

if not os.path.exists(BASE_IDX):
    raise FileNotFoundError(f'Index enriched introuvable: {BASE_IDX}')

idx = pd.read_csv(BASE_IDX)

# Columns to prefer for features
PREFERRED_NUMERIC = ['temperature_celsius','humidity_percent','precipitation_mm','wind_speed_ms','urban_heat_island_intensity']

for _, entry in idx.iterrows():
    try:
        raw_fp = entry['raw_csv']
        if not isinstance(raw_fp, str) or not raw_fp or not os.path.exists(raw_fp):
            append_log(f"Skipping missing raw file for entry: {entry.get('pipeline','?')} {entry.get('window_center','?')}")
            continue

        # Build output folder for this window
        wtype = entry.get('window_type','unknown')
        wc = entry.get('window_center','nan')
        pipeline = entry.get('pipeline','X')
        tag = safe_tag(f"{wtype}_{wc}_{pipeline}")
        out_dir = os.path.join(OUT_BASE, tag)
        os.makedirs(out_dir, exist_ok=True)

        # Read raw data
        df = pd.read_csv(raw_fp)
        if df.empty:
            append_log(f"No rows in raw file {raw_fp}, skipped.")
            continue

        # Ensure city_key
        if 'city_key' not in df.columns:
            agg_cols = [c for c in ['city','country','latitude','longitude'] if c in df.columns]
            if agg_cols:
                df['city_key'] = df[agg_cols].astype(str).agg('_'.join, axis=1)
            else:
                df['city_key'] = 'unknown'

        # Determine numeric feature columns
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        # remove control columns
        numeric_cols = [c for c in numeric_cols if c not in ['year','month','ym','n_rows']]
        # prioritize preferred set if present
        pref_present = [c for c in PREFERRED_NUMERIC if c in numeric_cols]
        if pref_present:
            features = pref_present
        else:
            # fallback: top numeric columns by variance
            if numeric_cols:
                variances = df[numeric_cols].var(numeric_only=True).sort_values(ascending=False)
                features = variances.index.tolist()[:min(10, len(variances))]
            else:
                features = []

        # Prepare diagnostics per feature
        diag_rows = []
        for feat in features:
            col = df[feat]
            n = col.shape[0]
            n_nan = int(col.isna().sum())
            frac_nan = float(n_nan)/max(1,n)
            valid = col.dropna().astype(float)
            if valid.empty:
                median_v = np.nan
                mean_v = np.nan
                std_v = np.nan
                skew_v = np.nan
                top3 = []
                bottom3 = []
            else:
                median_v = float(valid.median())
                mean_v = float(valid.mean())
                std_v = float(valid.std(ddof=1))
                try:
                    skew_v = float(skew(valid))
                except Exception:
                    skew_v = np.nan
                # top-3 extremes (largest absolute deviation from median)
                dev = (valid - median_v).abs()
                top3_idx = dev.sort_values(ascending=False).head(3).index
                top3 = [(int(i), float(valid.loc[i])) for i in top3_idx]
                bottom3 = [(int(i), float(valid.loc[i])) for i in valid.sort_values().head(3).index]
            diag_rows.append({
                'feature': feat,
                'n': n,
                'n_nan': n_nan,
                'frac_nan': frac_nan,
                'mean': mean_v,
                'median': median_v,
                'std': std_v,
                'skew': skew_v,
                'top3_idx_val': ';'.join([f"{i}:{v}" for i,v in top3]),
                'bottom3_idx_val': ';'.join([f"{i}:{v}" for i,v in bottom3])
            })
        diag_df = pd.DataFrame(diag_rows)
        diag_csv = os.path.join(out_dir, f'diagnostics_features_{tag}.csv')
        diag_df.to_csv(diag_csv, index=False)

        # Per-city summary (n_obs, medians per feature)
        group = df.groupby('city_key')
        city_rows = []
        for city, g in group:
            row = {'city_key': city, 'n_obs': int(g.shape[0])}
            for feat in features:
                try:
                    row[f'median_{feat}'] = float(g[feat].median()) if feat in g.columns else np.nan
                    row[f'mean_{feat}'] = float(g[feat].mean()) if feat in g.columns else np.nan
                    row[f'frac_nan_{feat}'] = float(g[feat].isna().sum())/max(1, g.shape[0]) if feat in g.columns else np.nan
                except Exception:
                    row[f'median_{feat}'] = np.nan
                    row[f'mean_{feat}'] = np.nan
                    row[f'frac_nan_{feat}'] = np.nan
            city_rows.append(row)
        city_df = pd.DataFrame(city_rows).sort_values('n_obs', ascending=False)
        city_csv = os.path.join(out_dir, f'diagnostics_by_city_{tag}.csv')
        city_df.to_csv(city_csv, index=False)

        # Heatmap matrix: rows = cities (sorted), cols = features; use median per city (or mean)
        if features:
            heat_mat = city_df.set_index('city_key')[[f'median_{f}' for f in features]].replace([np.inf, -np.inf], np.nan)
            # If many cities, limit display order to top N by n_obs (but keep all saved)
            # Normalize for display: z-score per feature if possible, fallback to min-max
            heat = heat_mat.copy().astype(float)
            # compute zscore per column where std>0
            for col in heat.columns:
                col_vals = heat[col]
                if col_vals.dropna().shape[0] >= 2 and not np.isclose(col_vals.std(ddof=1), 0.0):
                    heat[col] = (col_vals - col_vals.mean()) / col_vals.std(ddof=1)
                else:
                    mn = col_vals.min()
                    mx = col_vals.max()
                    if pd.isna(mn) or pd.isna(mx) or np.isclose(mx, mn):
                        heat[col] = 0.0
                    else:
                        heat[col] = (col_vals - mn) / (mx - mn)
            # Plot heatmap
            plt.figure(figsize=(max(6, min(14, heat.shape[1])), max(6, min(30, heat.shape[0]*0.4))))
            sns.heatmap(heat, cmap='vlag', center=0, cbar_kws={'label':'normalized value'}, linewidths=0.25)
            plt.title(f'Heatmap medians per city - {tag}')
            plt.tight_layout()
            heat_png = os.path.join(out_dir, f'heatmap_cities_features_{tag}.png')
            plt.savefig(heat_png, dpi=200)
            plt.close()
        else:
            heat_png = None

        # Short summary markdown
        md_fp = os.path.join(out_dir, f'summary_{tag}.md')
        with open(md_fp, 'w', encoding='utf-8') as f:
            f.write(f'# Diagnostics pour fenêtre {tag}\n\n')
            f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
            f.write(f'- raw file: {raw_fp}\n')
            f.write(f'- features used ({len(features)}): {features}\n')
            f.write(f'- diagnostics features CSV: {os.path.basename(diag_csv)}\n')
            f.write(f'- per-city CSV: {os.path.basename(city_csv)}\n')
            if heat_png:
                f.write(f'- heatmap: {os.path.basename(heat_png)}\n')
            else:
                f.write('- heatmap: none (no numeric features)\n')
            f.write('\n## Observations rapides\n\n')
            # add a tiny textual top3 for each feature
            for rid, r in diag_df.iterrows():
                f.write(f"- **{r['feature']}**: n={int(r['n'])}; frac_NaN={r['frac_nan']:.3f}; median={r['median']}; mean={r['mean']}; skew={r['skew']}\n")
            f.write('\n')
        append_log(f'Produced diagnostics for window {tag} -> {out_dir} (rows={df.shape[0]}, features={len(features)})')
        print(f'Wrote diagnostics for {tag}: {diag_csv}, {city_csv}, heatmap={heat_png}')
    except Exception as e:
        append_log(f'Error processing entry {entry.get("pipeline","?")} {entry.get("window_center","?")}: {e}')
        print('Error for entry:', entry.get('window_center','?'), entry.get('pipeline','?'), e)

print('All diagnostics completed. Outputs under:', OUT_BASE)


C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:175: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'


Wrote diagnostics for 3yr_1984.0_A: results\temporal_cv_atypical_windows\diagnostics\3yr_1984.0_A\diagnostics_features_3yr_1984.0_A.csv, results\temporal_cv_atypical_windows\diagnostics\3yr_1984.0_A\diagnostics_by_city_3yr_1984.0_A.csv, heatmap=results\temporal_cv_atypical_windows\diagnostics\3yr_1984.0_A\heatmap_cities_features_3yr_1984.0_A.png


C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:175: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'


Wrote diagnostics for 3yr_1984.0_B: results\temporal_cv_atypical_windows\diagnostics\3yr_1984.0_B\diagnostics_features_3yr_1984.0_B.csv, results\temporal_cv_atypical_windows\diagnostics\3yr_1984.0_B\diagnostics_by_city_3yr_1984.0_B.csv, heatmap=results\temporal_cv_atypical_windows\diagnostics\3yr_1984.0_B\heatmap_cities_features_3yr_1984.0_B.png


C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:175: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'


...

C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:175: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'


Wrote diagnostics for annual_2016.0_A: results\temporal_cv_atypical_windows\diagnostics\annual_2016.0_A\diagnostics_features_annual_2016.0_A.csv, results\temporal_cv_atypical_windows\diagnostics\annual_2016.0_A\diagnostics_by_city_annual_2016.0_A.csv, heatmap=results\temporal_cv_atypical_windows\diagnostics\annual_2016.0_A\heatmap_cities_features_annual_2016.0_A.png


C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:175: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'


Wrote diagnostics for annual_2016.0_B: results\temporal_cv_atypical_windows\diagnostics\annual_2016.0_B\diagnostics_features_annual_2016.0_B.csv, results\temporal_cv_atypical_windows\diagnostics\annual_2016.0_B\diagnostics_by_city_annual_2016.0_B.csv, heatmap=results\temporal_cv_atypical_windows\diagnostics\annual_2016.0_B\heatmap_cities_features_annual_2016.0_B.png
Wrote diagnostics for annual_2016.0_C: results\temporal_cv_atypical_windows\diagnostics\annual_2016.0_C\diagnostics_features_annual_2016.0_C.csv, results\temporal_cv_atypical_windows\diagnostics\annual_2016.0_C\diagnostics_by_city_annual_2016.0_C.csv, heatmap=results\temporal_cv_atypical_windows\diagnostics\annual_2016.0_C\heatmap_cities_features_annual_2016.0_C.png
All diagnostics completed. Outputs under: results\temporal_cv_atypical_windows\diagnostics


C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:175: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\3763337873.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'


Cellule Python — créer un fichier résumé compact + package des fichiers clés à envoyer

La cellule :

parcourt les diagnostics déjà produits sous results/temporal_cv_atypical_windows/diagnostics ;

construit un CSV résumé compact par fenêtre (one‑row per window) avec : window_type, window_center, pipeline, n_rows, prop_obs_outlier, median_md, prop_cities_with_gt10pct_outliers, max_city_outlier_frac, flag_for_review, top3_cities_by_outlier_frac, features_used, path_summary_md, path_heatmap, path_by_city_csv ;

crée un ZIP contenant pour chaque fenêtre les trois fichiers essentiels (summary markdown, by_city CSV, heatmap PNG quand présents) afin que vous puissiez m’envoyer un seul fichier compressé;

écrit le CSV résumé results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_summary.csv et le package results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_summary_package.zip.

In [17]:
# Création d'un résumé compact et package ZIP des fichiers clés à envoyer
import os, glob, zipfile, json, re
import pandas as pd
from datetime import datetime

BASE_IDX_ENR = 'results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index_enriched.csv'
DIAG_BASE = 'results/temporal_cv_atypical_windows/diagnostics'
OUT_SUMMARY_CSV = 'results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_summary.csv'
OUT_ZIP = 'results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_summary_package.zip'

def safe_tag(s):
    return re.sub(r'[^0-9A-Za-z_.-]', '_', str(s))

if not os.path.exists(BASE_IDX_ENR):
    raise FileNotFoundError(f'Fichier d’index enrichi introuvable: {BASE_IDX_ENR}')

idx = pd.read_csv(BASE_IDX_ENR)

rows = []
files_to_package = []

for _, r in idx.iterrows():
    wtype = r.get('window_type')
    wc = r.get('window_center')
    pipeline = r.get('pipeline')
    tag = safe_tag(f"{wtype}_{wc}_{pipeline}")
    diag_dir = os.path.join(DIAG_BASE, tag)
    # locate expected artifacts
    summary_md = glob.glob(os.path.join(diag_dir, f"summary_{tag}.md"))
    heatmap = glob.glob(os.path.join(diag_dir, f"heatmap_cities_features_{tag}.png"))
    by_city_candidates = glob.glob(os.path.join('results','temporal_cv_atypical_windows',f"*{tag}*_by_city.csv"))
    # choose paths or None
    summary_md_p = summary_md[0] if summary_md else None
    heatmap_p = heatmap[0] if heatmap else None
    by_city_p = by_city_candidates[0] if by_city_candidates else None
    # compute top3 cities by outlier frac if by_city available
    top3_cities = []
    if by_city_p and os.path.exists(by_city_p):
        try:
            bc = pd.read_csv(by_city_p)
            # detect columns that may indicate outlier fraction per city (looks for 'frac' or 'outlier')
            frac_cols = [c for c in bc.columns if 'outlier' in c.lower() or 'frac' in c.lower()]
            # fallback: use n_obs to rank
            if frac_cols:
                # use first frac column
                col = frac_cols[0]
                top3 = bc.sort_values(col, ascending=False).head(3)
                top3_cities = [f"{row['city_key']}:{row[col]:.3f}" for _, row in top3.iterrows()]
            else:
                top3 = bc.sort_values('n_obs', ascending=False).head(3)
                top3_cities = [f"{row['city_key']}:nobs={int(row['n_obs'])}" for _, row in top3.iterrows()]
        except Exception:
            top3_cities = []
    # features used: try to read diagnostics_features file
    feat_file = glob.glob(os.path.join(diag_dir, f"diagnostics_features_{tag}.csv"))
    features_used = None
    if feat_file:
        try:
            df_feat = pd.read_csv(feat_file[0])
            features_used = ';'.join(df_feat['feature'].astype(str).tolist())
        except Exception:
            features_used = None

    row = {
        'window_type': wtype,
        'window_center': wc,
        'pipeline': pipeline,
        'n_rows': int(r.get('n_rows', 0)) if not pd.isna(r.get('n_rows', None)) else None,
        'prop_obs_outlier': r.get('prop_obs_outlier'),
        'median_md': r.get('median_md'),
        'prop_cities_with_gt10pct_outliers': r.get('prop_cities_with_gt10pct_outliers'),
        'max_city_outlier_frac': r.get('max_city_outlier_frac'),
        'flag_for_review': bool(r.get('flag_for_review', False)),
        'top3_cities_by_outlier_or_nobs': ';'.join(top3_cities),
        'features_used': features_used,
        'path_summary_md': summary_md_p,
        'path_heatmap_png': heatmap_p,
        'path_by_city_csv': by_city_p
    }
    rows.append(row)

    # add files to package if exist (one set per window)
    to_add = []
    if row['path_summary_md']:
        to_add.append(row['path_summary_md'])
    if row['path_by_city_csv']:
        to_add.append(row['path_by_city_csv'])
    if row['path_heatmap_png']:
        to_add.append(row['path_heatmap_png'])
    # add raw index entry and raw CSV referenced in base index as well
    if 'raw_csv' in r and isinstance(r['raw_csv'], str) and os.path.exists(r['raw_csv']):
        to_add.append(r['raw_csv'])
    # deduplicate and extend files_to_package
    for p in to_add:
        if p and p not in files_to_package:
            files_to_package.append(p)

# create summary dataframe and save CSV
summary_df = pd.DataFrame(rows)
os.makedirs(os.path.dirname(OUT_SUMMARY_CSV), exist_ok=True)
summary_df.to_csv(OUT_SUMMARY_CSV, index=False)

# package files into zip
if files_to_package:
    with zipfile.ZipFile(OUT_ZIP, 'w', compression=zipfile.ZIP_DEFLATED) as z:
        for fp in files_to_package:
            arcname = os.path.join('atypical_windows_files', os.path.basename(fp))
            try:
                z.write(fp, arcname=arcname)
            except Exception:
                # skip files that cannot be read
                pass

print("Résumé CSV écrit :", OUT_SUMMARY_CSV)
print("Package ZIP écrit :", OUT_ZIP)
print(f"Fichiers inclus dans le package: {len(files_to_package)} (exemple 10):")
print(files_to_package[:10])


Résumé CSV écrit : results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_summary.csv
Package ZIP écrit : results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_summary_package.zip
Fichiers inclus dans le package: 57 (exemple 10):
['results/temporal_cv_atypical_windows/diagnostics\\3yr_1984.0_A\\summary_3yr_1984.0_A.md', 'results/temporal_cv_atypical_windows/diagnostics\\3yr_1984.0_A\\heatmap_cities_features_3yr_1984.0_A.png', 'results\\temporal_cv_atypical_windows\\temporal_cv_atypical_period_window_3yr_1984.00_A_raw.csv', 'results/temporal_cv_atypical_windows/diagnostics\\3yr_1984.0_B\\summary_3yr_1984.0_B.md', 'results/temporal_cv_atypical_windows/diagnostics\\3yr_1984.0_B\\heatmap_cities_features_3yr_1984.0_B.png', 'results\\temporal_cv_atypical_windows\\temporal_cv_atypical_period_window_3yr_1984.00_B_raw.csv', 'results/temporal_cv_atypical_windows/diagnostics\\3yr_1984.0_C\\summary_3yr_1984.0_C.md', 'results/temporal_cv_atypical_windows/diagnostics\\3yr_1984

Liste prioritaire détaillée — Pipeline A (A_baseline_StandardScaler)
Fichiers clés et intégrité

Vérifier : existence et lecture de results/pipeline_details/A_baseline_StandardScaler_d_estimate.csv, _loo.csv, _loo_summary.csv et results/tlog_pipelines_overview.csv.

Objectif : garantir que la sortie a bien été écrite et qu’elle n’est pas vide.

Cohérence des paramètres d_est / n_cities / T_log

Vérifier : la ligne dans A_baseline_StandardScaler_d_estimate.csv contient d_part, d_pca90, d_est, n_cities.

Critère prioritaire : |d_est_reporté - d_est_calculé_local| < 1e-6; n_cities == 20 (ou valeur attendue).

Robustesse LOO

Vérifier : existence de A_baseline_StandardScaler_loo.csv; calculer mean(T_log_loo), std et rel_std_pct.

Critère : rel_std_pct attendu ≈ 24.36% (tolérance configurable, p.ex.. ±2%), mean attendu ≈ -0.353046 (tolérance configurable).

Test statistique p-value

Vérifier : valeur tstat / pvalue dans A_baseline_StandardScaler_loo_summary.csv ou recomputer t-test; p-value << 0 indique rejet H0.

Priorité : haut (valide la signification statistique du résultat).

Sanitation / NaN / Inf

Vérifier : fichiers sanitized présents (results/tlog_d_estimates_sanitized.csv, results/tlog_leave_one_out_sanitized.csv) et absence de NaN/Inf dans colonnes numériques critiques (d_estimate, T_log, n_used).

Priorité : critique (évite instabilités downstream).

Sweep / Sensibilité

Vérifier : existence de results/{A_baseline_StandardScaler}_sweep_summary.csv; résumé fractions match_frac/unstable_frac conforme aux valeurs attendues.

Objectif : assurer comportement stable sous sous-échantillonnage.

Temporal CV / Fenêtres atypiques

Vérifier : si temporal CV a exporté fenêtres atypiques listées dans temporal_cv_atypical_periods.csv et index enriched.

Priorité : moyen (confirme export diagnostic pour revue).

Diagnostics features et qualité

Vérifier : existence de results/feature_quality_stats.csv et résultats d’outliers (feature_outliers_detected.csv).

Objectif : garantir que les features utilisées dans le calcul d sont saines.

Reproductibilité minimale

Vérifier : existence de results/params.json et README_method.md; exécution idempotente (re-lancer le script bloc 1 ne casse pas les fichiers).

Priorité : moyen (audit et traçabilité).

Plaque tournante d’alerte (flag review)

Vérifier : si windows_index_enriched flag_for_review == True pour fenêtres listées (p.ex. 3yr_1984.0_A).

Objectif : signaler les périodes à investiguer manuellement.



Remarques rapides

Ajustez EXPECTED_N_CITIES et tolérances (TOL_D, TOL_MEAN) selon votre jeu de données ou version de référence.

Cette cellule est pensée comme un smoke-test minimal, exécutable immédiatement ; pour audit complet, ajoutez assertions sur distributions des features, comparaison des spectres d’eigenvalues, et tests de reproductibilité (re-run deterministic).

In [18]:
# Cellule de tests rapides pour Pipeline A (A_baseline_StandardScaler)
# Execution: coller dans une cellule code et exécuter.
import os
import json
import pandas as pd
import numpy as np
from scipy import stats

# Configurable
TOL_D = 1e-6
TOL_MEAN = 1e-3
EXPECTED_N_CITIES = 20

# Chemins
base = "results/pipeline_details"
f_d = os.path.join(base, "A_baseline_StandardScaler_d_estimate.csv")
f_loo = os.path.join(base, "A_baseline_StandardScaler_loo.csv")
f_loo_summary = os.path.join(base, "A_baseline_StandardScaler_loo_summary.csv")
overview = "results/tlog_pipelines_overview.csv"
sanitized_summary = "results/tlog_d_estimates_sanitized.csv"
sanitized_loo = "results/tlog_leave_one_out_sanitized.csv"
sweep_summary = os.path.join(base, "A_baseline_StandardScaler_sweep_summary.csv")
temporal_index = "results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index_enriched.csv"

errors = []

def must_exist(path):
    if not os.path.exists(path):
        errors.append(f"MISSING: {path}")

must_exist(f_d)
must_exist(f_loo)
must_exist(f_loo_summary)
must_exist(overview)
must_exist(sanitized_summary)
must_exist(sanitized_loo)
# sweep and temporal index optional but flagged
if not os.path.exists(sweep_summary):
    print("Warning: sweep_summary absent (not fatal for minimal checks)")
if not os.path.exists(temporal_index):
    print("Warning: temporal_index absent (no atypical window exports found)")

# Read and basic checks
if os.path.exists(f_d):
    df_d = pd.read_csv(f_d)
    if df_d.empty:
        errors.append("D_EST file empty")
    else:
        row = df_d.iloc[-1]
        # Expect columns
        for col in ["d_part","d_pca90","d_est","n_cities","T_log"]:
            if col not in df_d.columns:
                # some files may not include T_log column; handle gracefully
                if col == "T_log" and "T_log" not in df_d.columns:
                    pass
                else:
                    errors.append(f"Column missing in d_estimate file: {col}")
        # Consistency: compare with overview if present
        if os.path.exists(overview):
            ov = pd.read_csv(overview)
            row_ov = ov[ov['pipeline'].str.contains("A_baseline_StandardScaler", na=False)]
            if not row_ov.empty:
                d_est_report = float(row_ov.iloc[0]["d_est"])
                d_est_file = float(row["d_est"])
                if abs(d_est_report - d_est_file) > TOL_D:
                    errors.append(f"d_est mismatch overview vs file: {d_est_report} vs {d_est_file}")

        # n_cities check
        n_cities = int(row.get("n_cities", EXPECTED_N_CITIES))
        if n_cities != EXPECTED_N_CITIES:
            errors.append(f"n_cities unexpected: {n_cities} (expected {EXPECTED_N_CITIES})")

# LOO checks
if os.path.exists(f_loo):
    loo = pd.read_csv(f_loo)
    if loo.empty:
        errors.append("LOO file empty")
    else:
        if "T_log" not in loo.columns:
            errors.append("LOO missing T_log column")
        else:
            meanT = loo["T_log"].mean()
            stdT = loo["T_log"].std(ddof=1)
            rel_std_pct = (stdT / (abs(meanT) + 1e-12)) * 100.0
            print(f"LOO mean T_log = {meanT:.6f}, std = {stdT:.6f}, rel_std_pct = {rel_std_pct:.2f}%")
            # Compare with expected summary if present
            if os.path.exists(f_loo_summary):
                summ = pd.read_csv(f_loo_summary)
                if "mean_T_LOO" in summ.columns:
                    expected_mean = float(summ.iloc[0]["mean_T_LOO"])
                    if abs(expected_mean - meanT) > TOL_MEAN:
                        errors.append(f"LOO mean mismatch summary vs recompute: {expected_mean} vs {meanT:.6f}")
            # t-test (recompute)
            tstat, pvalue = stats.ttest_1samp(loo["T_log"].dropna(), 0.0, alternative='two-sided')
            print(f"Recomputed t-test: t={tstat:.4f}, p={pvalue:.6e}")

# Sanitized files check for NaN/Inf
for p in [sanitized_summary, sanitized_loo]:
    if os.path.exists(p):
        df = pd.read_csv(p)
        numcols = df.select_dtypes(include=[np.number]).columns
        if df[numcols].isna().sum().sum() > 0:
            errors.append(f"NaN found in numeric columns after sanitization in {p}")
        if np.isinf(df[numcols].to_numpy()).sum() > 0:
            errors.append(f"Inf found in numeric columns after sanitization in {p}")

# Basic sweep check
if os.path.exists(sweep_summary):
    s = pd.read_csv(sweep_summary)
    if not {"fraction","match_frac","unstable_frac"}.issubset(s.columns):
        errors.append("Sweep summary missing expected columns")

# Temporal index check (flag_for_review)
if os.path.exists(temporal_index):
    t = pd.read_csv(temporal_index)
    if "flag_for_review" in t.columns:
        flagged = t[t["flag_for_review"]==True]
        print(f"Temporal CV: {len(flagged)} windows flagged for review")
    else:
        print("Temporal index present but no flag_for_review column")

# Final report
if errors:
    print("\nTESTS FAIL - issues list:")
    for e in errors:
        print(" -", e)
    raise AssertionError("One or more checks failed (see list above).")
else:
    print("\nAll checks passed (minimal pipeline A smoke tests).")


LOO mean T_log = -0.353046, std = 0.086006, rel_std_pct = 24.36%
Recomputed t-test: t=-18.3577, p=1.502880e-13
Temporal CV: 19 windows flagged for review

All checks passed (minimal pipeline A smoke tests).


Interprétation courte des résultats actuels

LOO mean T_log = -0.353046 avec std = 0.086006 indique un effet stable mais la relative std 24.36% montre sensibilité locale importante.

Le t-test rejette H0 (p ≈ 1.5e-13) — l’effet T_log est statistiquement significatif.

19 fenêtres temporelles sont marquées pour review ; plusieurs fenêtres montrent une ville avec max_city_outlier_frac ≈ 0.48–0.54.

D’après diagnostics globaux, les features les plus suspectes sont precipitation_mm et urban_heat_island_intensity ; les grandes villes (ex. New York, Los Angeles, Chicago, Houston, Phoenix, Philadelphia) figurent dans les séries sauvegardées et méritent une inspection prioritaire.

Liste prioritaire détaillée d’investigation (ordre d’exécution)
Identifier pour chaque fenêtre la ville ayant la plus grande max_city_outlier_frac (single top offender).

Pour ces villes (top offender par fenêtre), examiner les features responsables en regardant la proportion d’outliers par feature et les top‑3 indices d’outliers.

Vérifier si precipitation_mm et urban_heat_island_intensity concentrent les outliers ; si oui, prioriser : a) winsorisation, b) log1p sur precipitation_mm, c) vérifier erreur d’enregistrement / unité.

Effectuer influence test LOO‑city : recalculer T_log en excluant la ville top‑offender et mesurer delta T_log. Prioriser villes avec |delta T_log| élevé.

Pour chaque ville priorisée, produire plots : timeseries, box+points, et scatter (feature vs MD) pour détecter mécanismes (sauts, erreurs de capteur, vraie variabilité).

Après corrections (winsor / transform / exclusion conditionnelle), recomputer les métriques clefs : prop_obs_outlier, max_city_outlier_frac, LOO mean T_log, rel_std_pct, p‑value. Chercher prop_obs_outlier < 5% et max_city_outlier_frac < 25% comme objectif.

Cellule Python — extraction prioritaire villes/diagnostics + tests applicables (winsor/log1p/exclude-city LOO)
Collez et exécutez dans le notebook. La cellule :

lit l’index enrichi et les CSV by_city/raw listés ;

calcule, par fenêtre, la ville top‑offender (max fraction d’outliers) et la fraction d’outliers par feature pour cette ville ;

exécute trois tests rapides sur la fenêtre ciblée (winsor 1–99% sur precipitation_mm, log1p(precipitation_mm), exclusion de la ville top‑offender) et recalcule T_log approximé (utilise la même logique d_estimate moyenne d_participation/PCA90 et T_log = (d-4)*ln(n_eff)) ;

sauvegarde un CSV résumé tests : results/temporal_cv_atypical_windows/diagnostics/tests_priority_summary.csv et crée plots pour la fenêtre testée.

In [19]:
# Priority extraction + quick sensitivity tests (winsor / log1p / exclude-city LOO)
import os, re, glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

IDX = 'results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index_enriched.csv'
OUT_SUM = 'results/temporal_cv_atypical_windows/diagnostics/tests_priority_summary.csv'
OUT_DIR = 'results/temporal_cv_atypical_windows/diagnostics/priority_tests'
os.makedirs(OUT_DIR, exist_ok=True)

def safe_tag(s): return re.sub(r'[^0-9A-Za-z_.-]', '_', str(s))

def compute_d_estimate_from_X(X):
    # X: cities x features numeric (no NaN)
    if X.shape[0] < 2 or X.shape[1] < 1:
        return np.nan, np.nan
    cov = np.cov(X, rowvar=False)
    eigvals = np.linalg.eigvalsh(cov)
    eigvals = np.maximum(eigvals, 0.0)
    sum_eig = np.sum(eigvals)
    d_part = 0.0 if sum_eig <= 0 else (sum_eig**2) / np.sum(eigvals**2)
    pca = PCA(n_components=min(X.shape[0], X.shape[1])).fit(X)
    cumvar = np.cumsum(pca.explained_variance_ratio_)
    d_pca90 = int(np.searchsorted(cumvar, 0.90) + 1) if cumvar[-1] >= 0.90 else pca.n_components_
    d_est = float((d_part + d_pca90)/2.0)
    return d_part, d_pca90, d_est

def compute_T_log(d_est, n_eff):
    n_eff = max(2, int(n_eff))
    return (d_est - 4.0) * np.log(n_eff)

if not os.path.exists(IDX):
    raise FileNotFoundError(IDX)

idx = pd.read_csv(IDX)
summary_rows = []

# Prefer features list discovered in diagnostics/features files or use default list
DEFAULT_FEATURES = ['temperature_celsius','humidity_percent','precipitation_mm','wind_speed_ms','urban_heat_island_intensity']

for _, r in idx.iterrows():
    raw = r['raw_csv']
    by_city = r.get('city_summary_csv', None)
    tag = safe_tag(f"{r['window_type']}_{r['window_center']}_{r['pipeline']}")
    row_out = {'tag': tag, 'window_type': r['window_type'], 'window_center': r['window_center'],
               'pipeline': r['pipeline'], 'n_rows': int(r.get('n_rows', 0)),
               'prop_obs_outlier': r.get('prop_obs_outlier'), 'median_md': r.get('median_md'),
               'prop_cities_with_gt10pct_outliers': r.get('prop_cities_with_gt10pct_outliers'),
               'max_city_outlier_frac': r.get('max_city_outlier_frac'), 'flag_for_review': r.get('flag_for_review')}
    if not by_city or not os.path.exists(by_city):
        # try to find by_city in diagnostics folder
        candidate = glob.glob(os.path.join('results','temporal_cv_atypical_windows','diagnostics', tag, f"diagnostics_by_city_{tag}.csv"))
        by_city = candidate[0] if candidate else None
    if not raw or not os.path.exists(raw):
        summary_rows.append(row_out)
        continue
    df_raw = pd.read_csv(raw)
    # Ensure city_key
    if 'city_key' not in df_raw.columns:
        agg_cols = [c for c in ['city','country','latitude','longitude'] if c in df_raw.columns]
        df_raw['city_key'] = df_raw[agg_cols].astype(str).agg('_'.join, axis=1) if agg_cols else 'unknown'
    # detect numeric feature columns
    num_cols = [c for c in df_raw.select_dtypes(include=[np.number]).columns.tolist() if c not in ['year','month','n_rows']]
    features = [c for c in DEFAULT_FEATURES if c in num_cols] or (num_cols[:5] if num_cols else [])
    row_out['features_used'] = ';'.join(features)
    # if by_city exists, read per-city summary to identify top offender
    top_city = None
    top_frac = np.nan
    per_city_df = None
    if by_city and os.path.exists(by_city):
        per_city_df = pd.read_csv(by_city)
        # detect possible outlier fraction cols
        frac_cols = [c for c in per_city_df.columns if 'outlier' in c.lower() or 'frac' in c.lower()]
        if frac_cols:
            # take first frac-like column
            fcol = frac_cols[0]
            per_city_df = per_city_df.sort_values(fcol, ascending=False)
            top_city = per_city_df.iloc[0]['city_key']
            top_frac = float(per_city_df.iloc[0].get(fcol, np.nan))
        else:
            # fallback: use n_obs to pick representative city
            per_city_df = per_city_df.sort_values('n_obs', ascending=False)
            top_city = per_city_df.iloc[0]['city_key']
            top_frac = np.nan
    else:
        # compute naive per-city outlier fraction using Mahalanobis MD if present
        if 'mahalanobis_md' in df_raw.columns:
            g = df_raw.groupby('city_key')
            city_fracs = {}
            md = df_raw['mahalanobis_md'].values
            thresh = np.nanpercentile(md, 97.5)
            is_out = md >= thresh
            for city, grp in g:
                inds = grp.index.to_numpy()
                city_fracs[city] = np.nansum(is_out[inds]) / max(1, inds.size)
            if city_fracs:
                top_city = max(city_fracs, key=city_fracs.get)
                top_frac = city_fracs[top_city]
            per_city_df = pd.DataFrame([{'city_key':k,'outlier_frac':v} for k,v in city_fracs.items()]).sort_values('outlier_frac', ascending=False)
    row_out['top_city'] = top_city
    row_out['top_city_frac'] = top_frac

    # If top_city available, compute per-feature fraction of extreme rows in that city
    feat_out_fracs = {}
    if top_city is not None:
        city_rows = df_raw[df_raw['city_key']==top_city]
        for feat in features:
            if feat in city_rows.columns:
                col = city_rows[feat].dropna().astype(float)
                if col.empty:
                    feat_out_fracs[feat] = np.nan
                else:
                    # extreme using empirical 97.5% across entire raw feature distribution
                    global_thresh_hi = np.nanpercentile(df_raw[feat].dropna(), 97.5)
                    global_thresh_lo = np.nanpercentile(df_raw[feat].dropna(), 2.5)
                    n_ext = ((col >= global_thresh_hi) | (col <= global_thresh_lo)).sum()
                    feat_out_fracs[feat] = float(n_ext) / max(1, col.shape[0])
        row_out['top_city_feat_outlier_fracs'] = ';'.join([f"{k}:{v:.3f}" for k,v in feat_out_fracs.items()])
    else:
        row_out['top_city_feat_outlier_fracs'] = ''

    # Quick sensitivity tests: construct per-city aggregate X (mean per city), standardize, compute d_est and T_log baseline
    city_df = df_raw.groupby('city_key')[features].mean().reset_index()
    X = city_df[features].dropna().astype(float).values
    if X.size == 0 or X.shape[0] < 2:
        row_out.update({'d_part':np.nan,'d_pca90':np.nan,'d_est':np.nan,'T_log':np.nan})
        summary_rows.append(row_out); continue
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    d_part, d_pca90, d_est = compute_d_estimate_from_X(Xs)
    T_log_base = compute_T_log(d_est, Xs.shape[0])
    row_out.update({'d_part':d_part,'d_pca90':d_pca90,'d_est':d_est,'T_log':T_log_base})

    # Test 1: winsorize precipitation_mm at 1-99% (if present)
    test_results = {}
    if 'precipitation_mm' in features:
        df_w = df_raw.copy()
        lo, hi = np.nanpercentile(df_w['precipitation_mm'], [1,99])
        df_w['precipitation_mm'] = df_w['precipitation_mm'].clip(lo, hi)
        city_df_w = df_w.groupby('city_key')[features].mean().reset_index()
        Xw = city_df_w[features].dropna().astype(float).values
        if Xw.shape[0] >= 2:
            Xs_w = scaler.fit_transform(Xw)
            _,_, d_est_w = compute_d_estimate_from_X(Xs_w)
            T_w = compute_T_log(d_est_w, Xs_w.shape[0])
            test_results['winsor_precip_T_log'] = T_w
        else:
            test_results['winsor_precip_T_log'] = np.nan
    else:
        test_results['winsor_precip_T_log'] = np.nan

    # Test 2: log1p transform precipitation_mm
    if 'precipitation_mm' in features:
        df_l = df_raw.copy()
        df_l['precipitation_mm'] = np.log1p(df_l['precipitation_mm'].clip(lower=0.0))
        city_df_l = df_l.groupby('city_key')[features].mean().reset_index()
        Xl = city_df_l[features].dropna().astype(float).values
        if Xl.shape[0] >= 2:
            Xs_l = scaler.fit_transform(Xl)
            _,_, d_est_l = compute_d_estimate_from_X(Xs_l)
            T_l = compute_T_log(d_est_l, Xs_l.shape[0])
            test_results['log1p_precip_T_log'] = T_l
        else:
            test_results['log1p_precip_T_log'] = np.nan
    else:
        test_results['log1p_precip_T_log'] = np.nan

    # Test 3: exclude top_city and recompute (city influence)
    if top_city is not None:
        city_df_ex = city_df[city_df['city_key'] != top_city]
        Xex = city_df_ex[features].dropna().astype(float).values
        if Xex.shape[0] >= 2:
            Xs_ex = scaler.fit_transform(Xex)
            _,_, d_est_ex = compute_d_estimate_from_X(Xs_ex)
            T_ex = compute_T_log(d_est_ex, Xs_ex.shape[0])
            test_results['exclude_topcity_T_log'] = T_ex
            test_results['delta_T_log_exclude_topcity'] = T_ex - T_log_base
        else:
            test_results['exclude_topcity_T_log'] = np.nan
            test_results['delta_T_log_exclude_topcity'] = np.nan
    else:
        test_results['exclude_topcity_T_log'] = np.nan
        test_results['delta_T_log_exclude_topcity'] = np.nan

    # attach tests to row
    for k,v in test_results.items():
        row_out[k] = float(v) if (v is not None and not pd.isna(v)) else np.nan

    # Save quick plots for this window (top city timeseries + box for precipitation)
    try:
        fig_dir = os.path.join(OUT_DIR, tag)
        os.makedirs(fig_dir, exist_ok=True)
        if top_city is not None:
            df_city = df_raw[df_raw['city_key']==top_city]
            if 'precipitation_mm' in df_city.columns:
                plt.figure(figsize=(8,3))
                plt.plot(df_city['precipitation_mm'].values, marker='o', linestyle='-', markersize=3)
                plt.title(f'{tag} - precipitation_mm timeseries - top city {top_city}')
                plt.tight_layout()
                plt.savefig(os.path.join(fig_dir, f"{tag}_topcity_precip_timeseries.png"), dpi=150)
                plt.close()
            # boxplot of features for top city vs global
            for feat in features:
                plt.figure(figsize=(6,3))
                data = [df_raw[feat].dropna().values, df_city[feat].dropna().values]
                plt.boxplot(data, labels=['global','top_city'])
                plt.title(f'{tag} - box {feat} global vs {top_city}')
                plt.tight_layout()
                plt.savefig(os.path.join(fig_dir, f"{tag}_box_{feat}.png"), dpi=150)
                plt.close()
    except Exception:
        pass

    summary_rows.append(row_out)

# write summary CSV
pd.DataFrame(summary_rows).to_csv(OUT_SUM, index=False)
print("Priority tests summary written to:", OUT_SUM)
print("Per-window plots saved under:", OUT_DIR)


Priority tests summary written to: results/temporal_cv_atypical_windows/diagnostics/tests_priority_summary.csv
Per-window plots saved under: results/temporal_cv_atypical_windows/diagnostics/priority_tests
